**Import Necessari**

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

**Funzioni Utilizzate**

In [ ]:
def generator(dataset,batchsize,moves=10):
  while True:
    rand = np.random.randint(dataset.shape[0],size=batchsize)
    y_truth = dataset[rand]
    blurred = np.copy(y_truth)
    moving = tf.expand_dims(y_truth,axis=-1)

    for i in range(moves):
      #RandomTranslation requires a channel axis
      moving = layers.RandomTranslation(0.07,0.07,fill_mode='constant',interpolation='bilinear')(moving)
      blurred = blurred + tf.squeeze(moving)
    blurred = blurred/(moves+1)
    yield(blurred,y_truth)

def plot(x,y):
  no = x.shape[0]
  print(no)
  plt.figure(figsize=(2*no, 4))
  for i in range(no):
    # display original
    ax = plt.subplot(2, no, i + 1)
    plt.imshow(x[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, no, i + 1 + no)
    plt.imshow(y[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  plt.show()

def mean_square_error(x,x_true):
  summation = 0  #variable to store the summation of differences
  n = len(x) #finding total number of items in list
  for i in range (0,n):  #looping through each element of the list
    difference = x[i] - x_true[i]  #finding the difference between observed and predicted value
    squared_difference = difference**2  #taking square of the differene 
    summation = summation + squared_difference  #taking a sum of all the differences
  MSE = summation/n  #dividing summation by total values to obtain average
  print ("The Mean Square Error is: " , MSE)
  return MSE

def deblur_model(input_shape):
  inputs = layers.Input(shape=input_shape)
    
  # Encoder
  x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
  x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  
  # Decoder
  x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  outputs = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
  
  model = Model(inputs, outputs)
  return model


**Definizione del Modello**

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train/255.).astype(np.float32)
x_test = (x_test/255.).astype(np.float32)

x_train, x_valid = train_test_split(x_train, test_size=0.1, random_state=42)

**Fase di Training del Modello**

In [2]:
my_gen = generator(x_train,5)
test_x,test_y = next(my_gen)
plot(test_x,test_y)

batch_size = 32  # Puoi modificare la batch size come preferisci
epochs = 10
#steps_per_epoch = len(x_train)
#validation_steps = len(x_test)

model = deblur_model((28, 28, 1))
model.compile(optimizer='adam', loss='mse')

history = model.fit(
    generator(x_train, batch_size),
    epochs=epochs,
    steps_per_epoch=len(x_train) // batch_size,
    validation_data=(x_valid, x_valid)
)

#history = model.fit(
#    x_train, x_train,
#    epochs=epochs,
#    batch_size=batch_size,
#    validation_data=(x_valid, x_valid)
#)

#history = my_gen.fit(
#    generator(x_train, batch_size),
#   steps_per_epoch=steps_per_epoch,
#    epochs=20,  # Puoi aumentare il numero di epoche se necessario
#    validation_data=generator(x_test, batch_size),
#  validation_steps=validation_steps
#)

model.summary()

NameError: name 'generator' is not defined

**Fase di Validazione del Modello**

In [ ]:
test_loss = model.evaluate(generator(x_test, batch_size), steps=len(x_test) // batch_size)
#test_loss = model.evaluate(x_test, x_test)
print(f"Test Loss: {test_loss}")

plot(history,epochs)

#my_gen.fit( x_train, y_train, epochs=5, batch_size=5, validation_split=0.2 )
#mean_square_error(test_x,valid_x);

In [ ]:
#deblurred_images = model.predict(x_test)
deblurred_images = model.predict(generator(x_test, batch_size), steps=len(x_test) // batch_size)